In [1]:
import sys
sys.path.append("..\src")
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime

from sgcarmart_webscraper_functions import * # Imports all defined webscraping functions

In [2]:
# Creating a list of main pages to iterate through
main_page_listing_list = [] # creating list to store search pages of 100 car listings
for idx, link in enumerate(range(30)):
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(idx * 100) + "&RPG=100&AVL=2&VEH=2"
    main_page_listing_list.append(url)

print(main_page_listing_list,'\n','\n', len(main_page_listing_list))

['https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=200&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=300&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=400&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=500&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=600&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=700&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=800&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=900&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1000&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/li

In [3]:
# Base url, or you can think of this as the individual car listing prefix
base_url = 'https://www.sgcarmart.com/used_cars/'
listing_urls = []

# Acquiring indvidual car listings    
for main_link in main_page_listing_list:
   
    # Make a request to the website and get the object
    content = requests.get(main_link)

    # Parse the HTML text
    soup = BeautifulSoup(content.text, 'lxml')

    # Find every single URL in the webpage , refer to this post: # https://stackoverflow.com/questions/46490626/getting-all-links-from-a-page-beautiful-soup
    # This returns a list of every tag that contains a link in one main link (each element in main page listing)
    links = soup.find_all('a')
    
    # Create a list for storing all the individual listing urls
    
    for link in links:
        # Get link in <a href>
        suffix = link.get('href')

        # Check if 'ID=' and 'DL=' exist in the string
        if ('ID=' in suffix) and ('DL=' in suffix):

            # Concatenate the two strings if they do
            listing_url = base_url + suffix
            
            # Append result to the list
            listing_urls.append(listing_url)
            
#     Removing duplicates
    set_listing_urls = set(listing_urls)
    listing_urls = list(set_listing_urls)
    
    # Prevent oneself from getting blocked from the website
    time.sleep(1)

In [4]:
print(len(listing_urls))
print(len(set(listing_urls)))
print(len(list(set(listing_urls))))

3000
3000
3000


In [5]:
print(listing_urls[:10])

['https://www.sgcarmart.com/used_cars/info.php?ID=1084163&DL=1107', 'https://www.sgcarmart.com/used_cars/info.php?ID=1066944&DL=3034', 'https://www.sgcarmart.com/used_cars/info.php?ID=1090341&DL=2756', 'https://www.sgcarmart.com/used_cars/info.php?ID=1090029&DL=1123', 'https://www.sgcarmart.com/used_cars/info.php?ID=1075104&DL=1177', 'https://www.sgcarmart.com/used_cars/info.php?ID=1090701&DL=3570', 'https://www.sgcarmart.com/used_cars/info.php?ID=1080452&DL=3961', 'https://www.sgcarmart.com/used_cars/info.php?ID=1068625&DL=2976', 'https://www.sgcarmart.com/used_cars/info.php?ID=1072539&DL=4031', 'https://www.sgcarmart.com/used_cars/info.php?ID=1090240&DL=1360']


In [6]:
# Creating an empty DataFrame for attributes of interest
df = pd.DataFrame(columns=['LISTING_URL', 'BRAND', 'PRICE', 'DEPRE_VALUE_PER_YEAR',
       'REG_DATE', 'MILEAGE_KM', 'MANUFACTURED_YEAR',
       'ROAD_TAX_PER_YEAR','TRANSMISSION', 'DEREG_VALUE_FROM_SCRAPE_DATE',
       'SCRAPE_DATE', 'OMV', 'ARF', 'COE_FROM_SCRAPE_DATE',
       'DAYS_OF_COE_LEFT', 'ENGINE_CAPACITY_CC', 'CURB_WEIGHT_KG',
       'NO_OF_OWNERS', 'VEHICLE_TYPE'])

In [11]:
filename = 'sgcarmart_used_cars_prices'
i = 0 # Indexing rows in the DF

for listingurl in listing_urls:
    response = requests.get(listingurl)
    listing_url = BeautifulSoup(response.text, 'lxml')
    
    # Retrieval functions to pull data from the Individual Listings after they have been parsed
    df.loc[i, 'LISTING_URL'] = listingurl
    df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    df.loc[i, 'PRICE'] = price_retrieval(listing_url)
    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan
    
    try:
        df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan

    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan

        
    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan

    try:
        df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan
        
    try:
        df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    except:
        df.loc[i, 'CURB_WEIGHT_KG'] = np.nan
        
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
        
    try:
        df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
        
    df.to_csv("{}.csv".format(filename))    
        
    i += 1 # Allows next car listing to be put into a next row in the dataframe
    time.sleep(1)  # Prevents us from getting locked out of the website

In [12]:
df = pd.read_csv('sgcarmart_used_cars_prices.csv',index_col=0)
df

,LISTING_URL,BRAND,PRICE,DEPRE_VALUE_PER_YEAR,REG_DATE,MILEAGE_KM,MANUFACTURED_YEAR,ROAD_TAX_PER_YEAR,TRANSMISSION,DEREG_VALUE_FROM_SCRAPE_DATE,SCRAPE_DATE,OMV,ARF,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,CURB_WEIGHT_KG,NO_OF_OWNERS,VEHICLE_TYPE
0,https://www.sgcarmart.com/used_cars/info.php?I...,Lexus,150800.0,24390.0,23-Dec-2016,116000.0,2016,1210.0,Auto,76021.0,18/04/2022,56222.0,73200.0,54901.0,1704.0,1998.0,2070.0,1.0,SUV
1,https://www.sgcarmart.com/used_cars/info.php?I...,Nissan,43900.0,10280.0,28-Apr-2016,53000.0,2016,508.0,Auto,21826.0,18/04/2022,13026.0,5000.0,45504.0,1469.0,1198.0,1044.0,1.0,Hatchback
2,https://www.sgcarmart.com/used_cars/info.php?I...,BMW,52800.0,15500.0,17-Nov-2014,NaN,2014,684.0,Auto,33874.0,18/04/2022,31090.0,25526.0,72201.0,919.0,1499.0,1360.0,2.0,Hatchback
3,https://www.sgcarmart.com/used_cars/info.php?I...,Land,498800.0,44980.0,27-Jan-2021,31119.0,\nSUV,2382.0,2020,198735.0,18/04/2022,NaN,NaN,49001.0,3198.0,2996.0,2267.0,1.0,NaN
4,https://www.sgcarmart.com/used_cars/info.php?I...,Bentley,309000.0,33870.0,01-Sep-2012,67000.0,2012,7074.0,Auto,45357.0,18/04/2022,189986.0,189986.0,49716.0,3318.0,5998.0,2320.0,3.0,Sports Car
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,https://www.sgcarmart.com/used_cars/info.php?I...,Mercedes-Benz,238800.0,24650.0,19-Oct-2020,9770.0,2019,1202.0,Auto,78939.0,19/04/2022,47274.0,58184.0,41510.0,3079.0,1991.0,1555.0,1.0,Sports Car
2996,https://www.sgcarmart.com/used_cars/info.php?I...,Subaru,56900.0,13620.0,16-Sep-2015,NaN,2015,1798.0,Auto,34324.0,19/04/2022,20571.0,20800.0,61001.0,1222.0,2498.0,1659.0,1.0,SUV
2997,https://www.sgcarmart.com/used_cars/info.php?I...,Porsche,100800.0,15220.0,23-Oct-2000,NaN,2000,3999.0,Auto,21270.0,19/04/2022,67995.0,95193.0,32121.0,2401.0,3179.0,NaN,NaN,Sports Car
2998,https://www.sgcarmart.com/used_cars/info.php?I...,Volvo,71800.0,12960.0,25-Jan-2017,66688.0,2016,684.0,Auto,38217.0,19/04/2022,23513.0,19919.0,50889.0,1735.0,1498.0,1454.0,1.0,Hatchback
